Leon 1-1

Name topics
* sort tweets by highest coefficient for each topic
* look thru these tweets to determine topic name

* or have one column with the topic (being the highest coefficient)
    * can use this column to create a hue
    * bar charts ok

* no need to rearrange words

add bot_proba & determine threshold
* or, keep bot_proba & show avg bot_proba per topic!!!
    * box & whiskers

pull statistics
* x% of candidate ys tweets were bots
    * per topic
* what were bots tweeting/not tweeting about

In [1]:
!pip3 install pickle5
import pickle5 as pickle

import pandas as pd
import numpy as np

import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF, LatentDirichletAllocation

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import nltk
from nltk.corpus import stopwords as nltk_stop
from stop_words import stop_words as custom_stop

In [2]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.23.2.


In [3]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_colwidth', None)

In [4]:
data = pd.read_pickle("pickle/n2tk_limited.pick")
data.shape

(363310, 11)

In [5]:
data.columns

Index(['id', 'date', 'time', 'user_id', 'username', 'hashtags', 'trump',
       'biden', 'original', 'tweet', 'num_tokens'],
      dtype='object')

In [6]:
# with open("pickle/nmf_model.pick", 'rb') as f:
#     nmf_model = pickle.load( f)

with open("pickle/nmf_doc_topics30.pick", 'rb') as f:
    doc_topic = pickle.load( f)

with open("pickle/topic_words30.pick", 'rb') as f:
    topic_words = pickle.load( f)

In [7]:
doc_topic.shape

(363310, 30)

## Naming Topics

Before we can name our topics, we need to classify each tweet in our original DataFrame with its corresponding topic.

The values in our doc_topic matrix has rows=tweets, columns=topics, vaues=coefficients. We'll simply classify each tweet with the largest coefficient.

In [8]:
topic = doc_topic.argmax(axis=1)
data['tweet_topic'] = topic

In [9]:
# nmf_model.components_

In [10]:
# doc_topic

In [11]:
data.shape

(363310, 12)

In [12]:
doctop = pd.DataFrame(doc_topic)
doctop

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,0.000404,0.000130,0.000000,0.000775,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000270,0.001521,0.000162,0.000000,0.001254,0.000546,0.002367,0.000000,0.000132,0.000000,0.000841,0.000000,0.000073,0.000000,0.000261,0.000206,0.000000,0.000078,0.000107,0.000000,0.000448
1,0.000000,0.000000,0.000782,0.000142,0.000000,0.000472,0.000000,1.802203e-07,0.000343,0.000000,0.000198,0.001355,0.000423,0.000000,0.000000,0.000000,0.000319,0.000000,0.000000,0.000000,0.064040,0.001514,0.000000,0.000000,0.000011,0.000000,0.000694,0.000000,0.000515,0.000002
2,0.000000,0.000000,0.000000,0.000123,0.039796,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000021,0.000000,0.000000,0.000000,0.000000,0.000257,0.000000,0.000000,0.000000,0.000844,0.057241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000517
3,0.000425,0.000000,0.000000,0.046832,0.000000,0.000799,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000261,0.000000,0.000313,0.000000,0.000000,0.056653,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000490,0.023987,0.000000,0.000000,0.000000,0.000203,0.000489,0.000000e+00,0.000000,0.000000,0.009402,0.000000,0.032388,0.000000,0.000127,0.000000,0.000000,0.000000,0.000398,0.000000,0.000648,0.000000,0.000000,0.000000,0.000000,0.000090,0.000000,0.001582,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363305,0.001366,0.000000,0.000061,0.000000,0.002162,0.000482,0.000000,9.000112e-04,0.000000,0.000000,0.000752,0.002168,0.000000,0.035015,0.000000,0.001295,0.000000,0.000000,0.000000,0.000351,0.000000,0.000656,0.000010,0.000000,0.000000,0.002954,0.000000,0.001469,0.000000,0.000068
363306,0.000394,0.000000,0.000000,0.000297,0.000000,0.000341,0.000190,2.848399e-03,0.044850,0.000075,0.000178,0.000540,0.000000,0.000000,0.000000,0.022259,0.000000,0.000000,0.000000,0.000000,0.000500,0.001171,0.000207,0.005273,0.000000,0.000000,0.003612,0.000000,0.000195,0.013464
363307,0.000214,0.000006,0.003559,0.000281,0.000000,0.000257,0.000608,2.595221e-04,0.000000,0.000000,0.000000,0.002820,0.001422,0.000065,0.000000,0.000000,0.000000,0.000194,0.000651,0.000434,0.000000,0.040403,0.000000,0.000000,0.001162,0.001584,0.000000,0.011418,0.000424,0.001367
363308,0.001451,0.000000,0.000000,0.000463,0.000000,0.000000,0.000626,3.063218e-02,0.000000,0.000000,0.000248,0.000000,0.000599,0.000000,0.000000,0.000000,0.000000,0.000000,0.000293,0.000000,0.000000,0.000000,0.002310,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004221


Let's sort our tweets by topic coefficient, so that we can see which tweets have the highest coefficient for each topic. This will help us to identify the meaning of each topic.

Let's get the top ten tweets per topic.

In [13]:
# ensure topics start at 1 (and not zero)
if list(doctop.columns)[0] == 0:
    doctop.columns = doctop.columns + 1
    data.tweet_topic = data.tweet_topic + 1

# read topic tweets by updating topic_num
topic_num = 13
top10_ix = doctop[topic_num].sort_values(ascending=False).head(100).index # returns indexes of best tweets matching topic 1
data[['username', 'tweet', 'original', 'date']].iloc[top10_ix]


,username,tweet,original,date
560102,anthony76365973,joebiden vote thing away add lie donaldtrump,@JoeBiden I need to see more of your adds attacking Trump. All I see is his lies about you. I want to vote for you. Please dont let him get away with these things.,2020-10-18
70077,minnesotaken,trump need case get tic lie donaldtrump,@gregkellyusa @realDonaldTrump @PressSec @kayleighmcenany Everytime she gets ready to tell Trump's lies she needs a case of tic tac.,2020-10-01
205762,jlkaraoke,debate joebiden say as wont lie donaldtrump,Donald Trump does not have Covid19 it's another lie so he won't have to debate Biden and make an ass of himself again. Everything Trump says is a lie.,2020-10-04
777954,ileiarez,truth joebiden say maybe believe lie donaldtrump,@JoeBiden Maybe you should have told the truth before because the election is tomorrow and people believe everything trump says even though it’s all lies,2020-11-02
27898,alexisgrunder,truth joebiden babe blacklivesmatter know lie kamalaharris,From the mouthes of babes. Even at 7 he knows truth over lies and black lives matter! #JoeBiden2020 @JoeBiden #BLM @KamalaHarris https://t.co/33bhfdNDdE,2020-10-12
131300,marydug13569860,claim joebiden say get lie policy donaldtrump,'Everything he says is a lie': Trump claims Biden gets a pass on his policy flip-flops https://t.co/G5yXmMtHK5,2020-10-11
614203,purple54193932,hear taking u need give lie l donaldtrump,"@falifali67 @sarahcpr He needs to what? WIN?? HE ALREADY GIVES US WAY MORE LIES THAN WE WANT TO HEAR.. Trump is taking the big ""L"" Nov.3rd",2020-10-23
371150,quvester,joebiden race he ad lie video say,"@JoeBiden Its time for you to run your ads. Did you see the video? He says he's good and if it's a lie, IT IS WHAT IT IS. We have a race to run.",2020-10-03
264926,jamietre,perfected true seem good know lie donaldtrump,"@texas_much @SandraSentinel @jaketapper And yet still all Trump has to do is say ""it's not true"". Even the guy who has perfected the lie can't seem to do that this time, because he knows they have the goods.",2020-10-07
122301,resistanceblue1,trump joebiden lied v subtract time lie kamalaharris donaldtrump,"@omarirob55 @Bill_Thompson We can do that. They've lied about 8 times Now, i subtract that from Trump's total... I'm looking at 35 lies by Kamala and Biden vs... 20,218 for Trump https://t.co/4AmhnpHxYb",2020-10-11


## Final Topic Names

After looking through the topic_words.csv file and testing each of the topics in the above cell, here's what I came up with:

1. "anti-trump supporters",
2. "#vote",
3. "covid stats",
4. "vp candidates",
5. "#presidentialdebate",
6. "trump responsible for covid",
7. "past candidates",
8. "#hunterbiden",
9. "trump rally drama",
10. "#registertovote",
11. "#blm",
12. "covid policy",
13. "truth and lies",
14. "voter suppression/fraud",
15. "supreme court nom.",
16. "#economy",
17. "foreign affairs",
18. "#media",
19. "trump distrust science",
20. "#votebiden",
21. "#polling",
22. "healthcare for all",
23. "#factcheck",
24. "town halls",
25. "federal stimulous",
26. "#votetrump",
27. "trump covid diagnosis",
28. "#covidhoax",
29. "#breakingnews",
30. "trump rally covid infections"


In [14]:
data['topic'] = doc_topic.argmax(axis=1)
if np.min(data.topic.to_numpy()) == 0:
    data['topic'] += 1
    
topics = {
    1:"anti-trump supporters",
    2:"#vote",
    3:"covid stats",
    4:"vp candidates",
    5:"#presidentialdebate",
    6:"trump responsible for covid",
    7:"past candidates",
    8:"#hunterbiden",
    9:"trump rally drama",
    10:"#registertovote",
    11:"#blm",
    12:"covid policy",
    13:"negative campaign ads",
    14:"voter suppression/fraud",
    15:"supreme court nom.",
    16:"#economy",
    17:"foreign affairs",
    18:"#media",
    19:"trump distrust science",
    20:"#votebiden",
    21:"#polling",
    22:"healthcare for all",
    23:"#factcheck",
    24:"town halls",
    25:"federal stimulus",
    26:"#votetrump",
    27:"trump covid diagnosis",
    28:"#covidhoax",
    29:"#breakingnews",
    30:"trump rally covid infections"
}

data['topic'] = data['topic'].map(topics)
data.to_pickle("pickle/tweet_topics.pick")

In [15]:
data[['username', 'date', 'topic', 'original', 'tweet']].sample(10)

,username,date,topic,original,tweet
329669,bbqweed,2020-10-29,#economy,"@j46972435 @livesmattershow @lilpump @JoeBiden Wrong... because Biden says he will repeal all previous Trump tax cuts, which will then result in those who make under $400k also pay more taxes. 🤦‍♂️🤦‍♂️🤦‍♂️",joebiden say previous repeal result tax cut pay donaldtrump
153787,piaowens,2020-10-02,#votetrump,@Nonfinality Trump spent this year mocking people with COVID and urging Americans to ignore public health safety guidelines. No other imprisoned person has his power or platform. They haven't actively encouraged others to commit crimes and they have not held themselves out as world leaders.,urging actively platform ignore havent power guideline commit crime spent leader safety covid publichealth world donaldtrump
153716,serenitynow3121,2020-10-02,#economy,@augustayc @ivyzyan @Travis_Waldron @Robillard @AmyMcGrathKY I wasn’t really going to request my money back but when I contributed to her months ago I had no idea she supported Trump. I didn’t know that until today. It’s my fault for not doing my research.,fault research money request ago idea month donaldtrump
405672,elwould,2020-10-04,#presidentialdebate,@Deena_Zaru We know these doctors are lying through their teeth. We are not stupid. It's a ploy to sell more drugs. Because Trump has stock in it. Not to mention he's scared to do the next debate.,debate sell ploy doctor mention stock he teeth lying stupid drug donaldtrump
636568,rayandchay,2020-10-25,#registertovote,"@NikkiHaley Did you tell her that Trump was already President and her sign was endorsing Biden? Because Trump has been President for four years and you are saying it still sucks...Good lord, why do we need to explain this to you? Please, make America better by voting for Biden!",joebiden suck lord sign year explain voting donaldtrump
609723,keihud,2020-10-23,#presidentialdebate,Did Trump Say Wind Turbines Cause Cancer Like Biden Claimed in Presidential Debate? https://t.co/IjSDIWK1Zz,joebiden debate presidential cause cancer wind turbine donaldtrump
104689,brooksatnight,2020-10-07,federal stimulus,"@jaredthefox92 @Mag00631494 @benshapiro Also, I don’t know how Biden being a weird grandpa is worse than Trump taking pride on how and where he likes to grabs women or how he can’t help but admit he would love to date his daughter, if only they weren’t related -darn! https://t.co/h3rOrhWYFp",joebiden taking pride daughter date darn grandpa worse grab woman admit related help weird like donaldtrump
735857,facilitaor,2020-10-20,federal stimulus,"Just like he said about Biden.Give him the Rope,sit back and be quiet and he'll hang himself. He's getting very desperate knowing he's behind in the polls. His behaviors do not help win favor with the people.Not referring to his supporters. #Referringtothoseheneedstowintowin",behavior quiet desperate hell he knowing supporter poll getting help behind
460741,siliconvalleyex,2020-10-02,#covidhoax,@KamalaHarris Put the ads back up. You can’t let up even if Trump has COVID. He contracted the virus because of his own recklessness. We can’t afford to be our normal nice ppl. Too much at stake. @JoeBiden,joebiden virus nice afford normal ad contracted covid recklessness stake kamalaharris donaldtrump
668743,stevead13487346,2020-10-19,#factcheck,@grantstern Check out Mr Big Guy 10 percent A treasure trove of emails He should quit Biden????? A liar????,liar joebiden quit trove check treasure percent


In [16]:
data.groupby('topic').agg('count')

,id,date,time,user_id,username,hashtags,trump,biden,original,tweet,num_tokens,tweet_topic
topic,,,,,,,,,,,,
#blm,12400,12400,12400,12400,12400,12400,12400,12400,12400,12400,12400,12400
#breakingnews,10107,10107,10107,10107,10107,10107,10107,10107,10107,10107,10107,10107
#covidhoax,17061,17061,17061,17061,17061,17061,17061,17061,17061,17061,17061,17061
#economy,11790,11790,11790,11790,11790,11790,11790,11790,11790,11790,11790,11790
#factcheck,10328,10328,10328,10328,10328,10328,10328,10328,10328,10328,10328,10328
#hunterbiden,15507,15507,15507,15507,15507,15507,15507,15507,15507,15507,15507,15507
#media,6136,6136,6136,6136,6136,6136,6136,6136,6136,6136,6136,6136
#polling,16431,16431,16431,16431,16431,16431,16431,16431,16431,16431,16431,16431
#presidentialdebate,9213,9213,9213,9213,9213,9213,9213,9213,9213,9213,9213,9213
